<a href="https://colab.research.google.com/github/AbdoulayeDiop/lettuce-npk-deficiency/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
!pip install transformers datasets evaluate accelerate pillow torchvision scikit-learn mlflow
!git lfs install

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("baronn/lettuce-npk-dataset")

print("Path to dataset files:", path)

100%|██████████| 337M/337M [00:05<00:00, 67.6MB/s]


In [7]:
from datasets import load_dataset, Dataset
import os
dataset_path = os.path.join(path, "FNNPK")
dataset = load_dataset(dataset_path)
dataset = dataset["train"].train_test_split(test_size=0.1)

Resolving data files:   0%|          | 0/208 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
labels_names = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label_name in enumerate(labels_names):
    label2id[label_name] = i
    id2label[i] = label_name

In [9]:
class_elems = {id: [] for id in id2label}
for i, data in enumerate(dataset["train"]):
    class_elems[data["label"]].append(i)
print({k: len(v) for k, v in class_elems.items()})

In [10]:
import numpy as np
balanced_indices = []
for indices in class_elems.values():
    balanced_indices += list(np.random.choice(indices, size=500))
dataset["train"] = dataset["train"].select(balanced_indices)

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 21
    })
})

In [12]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [13]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor, RandomRotation

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), RandomRotation(180), ToTensor(), normalize])

In [14]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [15]:
dataset = dataset.with_transform(transforms)

In [16]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [17]:
import evaluate

accuracy = evaluate.load("accuracy")


In [18]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [19]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels_names),
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
import mlflow

os.environ["MLFLOW_EXPERIMENT_NAME"] = "lettuce-npk-deficiency-prediction"
os.environ["MLFLOW_FLATTEN_PARAMS"] = "1"

os.environ["WANDB_PROJECT"] = "lettuce-npk-deficiency-prediction"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

In [33]:
from datetime import datetime
training_args = TrainingArguments(
    hub_model_id="AbdoulayeDIOP/lettuce-npk-vit",
    report_to="wandb",  # enable logging to W&B
    run_name=datetime.now().strftime("%Y%m%d_%H%M"),  # name of the W&B run (optional)
    output_dir="./output",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [34]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    # processing_class=image_processor,
    compute_metrics=compute_metrics,
)

In [35]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy
0,1.163300,1.123938,0.857143
1,0.480200,0.421348,0.904762
2,0.138600,0.250051,0.952381
4,0.087100,0.348213,0.857143
5,0.070200,0.204820,0.952381
6,0.059400,0.282426,0.904762
8,0.039800,0.336282,0.857143
9,0.034800,0.251826,0.952381
10,0.041100,0.036894,1.000000
12,0.035300,0.236368,0.857143


wandb: Adding directory to artifact (./output/checkpoint-31)... Done. 8.6s
wandb: Adding directory to artifact (./output/checkpoint-62)... Done. 10.2s
wandb: Adding directory to artifact (./output/checkpoint-93)... Done. 11.0s
wandb: Adding directory to artifact (./output/checkpoint-125)... Done. 17.0s
wandb: Adding directory to artifact (./output/checkpoint-156)... Done. 18.3s
wandb: Adding directory to artifact (./output/checkpoint-187)... Done. 20.4s
wandb: Adding directory to artifact (./output/checkpoint-218)... Done. 11.3s
wandb: Adding directory to artifact (./output/checkpoint-250)... Done. 12.2s
wandb: Adding directory to artifact (./output/checkpoint-281)... Done. 20.4s
wandb: Adding directory to artifact (./output/checkpoint-312)... Done. 24.9s
wandb: Adding directory to artifact (./output/checkpoint-343)... Done. 27.7s
wandb: Adding directory to artifact (./output/checkpoint-375)... Done. 8.6s
wandb: Adding directory to artifact (./output/checkpoint-406)... Done. 19.8s
wand

TrainOutput(global_step=620, training_loss=0.14972498532264464, metrics={'train_runtime': 3843.7636, 'train_samples_per_second': 10.406, 'train_steps_per_second': 0.161, 'total_flos': 3.0749372662992077e+18, 'train_loss': 0.14972498532264464, 'epoch': 19.84})

In [36]:
mlflow.end_run()

In [37]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/AbdoulayeDIOP/lettuce-npk-vit/commit/7a36017377a17e6d61e5d606f10bd5511111bd0f', commit_message='End of training', commit_description='', oid='7a36017377a17e6d61e5d606f10bd5511111bd0f', pr_url=None, pr_revision=None, pr_num=None)

In [66]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-classification", model="AbdoulayeDIOP/lettuce-npk-vit")

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [67]:
pipe("https://cdn.britannica.com/77/170677-050-F7333D51/lettuce.jpg")

[{'label': '-N', 'score': 0.8333030939102173},
 {'label': 'FN', 'score': 0.767291247844696},
 {'label': '-K', 'score': 0.20579352974891663},
 {'label': '-P', 'score': 0.18466755747795105}]